Prepare stimuli shared across all analogy evaluations.

In [1]:
from collections import Counter, defaultdict
import functools
import pickle

import numpy as np
import pandas as pd
import torch
from tqdm.auto import tqdm

from src.analysis.state_space import StateSpaceAnalysisSpec
from src.analysis import analogy

In [2]:
state_space_specs_path = f"outputs/state_space_specs/librispeech-train-clean-100/w2v2_8/state_space_specs.h5"

pos_counts_path = "data/pos_counts.pkl"
output_dir = "."

seed = 1234

min_samples_per_word = 5
max_samples_per_word = 100

inflection_targets = [
    "VBD",
    "VBZ",
    "VBG",
    "NNS",
    "NOT-latin",
]

In [3]:
state_space_spec = StateSpaceAnalysisSpec.from_hdf5(state_space_specs_path, "word")
state_space_spec = state_space_spec.subsample_instances(max_samples_per_word)

In [4]:
with open(pos_counts_path, "rb") as f:
    pos_counts = pickle.load(f)

In [5]:
cuts_df = state_space_spec.cuts.xs("phoneme", level="level").drop(columns=["onset_frame_idx", "offset_frame_idx"])
cuts_df["label_idx"] = cuts_df.index.get_level_values("label").map({l: i for i, l in enumerate(state_space_spec.labels)})
cuts_df["frame_idx"] = cuts_df.groupby(["label", "instance_idx"]).cumcount()
cuts_df = cuts_df.reset_index().set_index(["label", "instance_idx", "frame_idx"]).sort_index()

In [6]:
cut_phonemic_forms = cuts_df.groupby(["label", "instance_idx"]).description.agg(' '.join)

In [7]:
ss_spans = state_space_spec.target_frame_spans_df

In [14]:
# import re
# s = """Condition Target Prime Test Con Test Con Test Con+ M+ S+ O vague vagueness 576 562 619 632 610 627+ M+ S+ O bleak bleakly 543 668 588 625 598 691+ M+ S+ O quick quickly 517 544 586 528 538 560+ M+ S+ O blunt bluntly 636 684 644 606 655 585+ M+ S+ O sad sadness 536 603 517 557 496 594+ M+ S+ O prison prisoner 567 613 499 625 568 571+ M+ S+ O farm farmer 530 592 486 555 521 587+ M+ S+ O sleep sleepless 565 523 535 544 497 660+ M+ S+ O fate fateful 563 604 614 480 584 570+ M+ S+ O cease ceaseless 673 668 634 718 659 688+ M+ S+ O blame blameless 541 545 576 597 551 682+ M+ S+ O bake baker 525 649 613 610 559 651+ M+ S+ O hunt hunter 559 684 503 629 523 574+ M+ S+ O swim swimmer 546 636 494 578 628 633+ M+ S+ O yellow yellowish 530 594 490 609 604 549+ M+ S+ O punish punishable 565 601 543 667 535 591+ M+ S+ O starve starvation 592 605 495 585 641 657+ M+ S+ O paint painter 518 595 516 609 507 597+ M+ S+ O dance dancer 531 529 485 549 528 604+ M+ S+ O discover discovery 634 636 464 638 584 606+ M+ S+ O confirm confirmation 535 615 518 626 578 636+ M+ S+ O depend dependent 559 586 548 629 578 582+ M+ S+ O rebel rebellious 574 581 496 636 553 598+ M+ S+ O select selection 547 647 490 609 545 595+ M±S+ O audit audition 649 600 674 612 732 712+ M±S+ O casual casualty 614 579 591 548 626 581+ M±S+ O compete competence 707 668 694 551 737 595+ M±S+ O design designate 529 563 545 531 598 593+ M±S+ O list listless 598 604 592 498 595 575+ M±S+ O organ organise 594 559 592 607 638 658+ M±S+ O ration rational 671 592 647 607 730 752+ M±S+ O seed seedy 518 552 510 555 515 570+ M±S+ O virtue virtual 552 616 658 597 705 757+ M±S+ O wit witness 576 674 647 603 783 649+ M±S+ O crank cranky 556 813 562 686 720 770+ M±S+ O blaze blazer 566 572 579 524 694 678+ M±S+ O depart department 564 573 511 600 605 620+ M±S+ O apart apartment 631 726 680 704 651 694+ M±S+ O awe awful 586 631 588 655 599 621+ M±S+ O fruit fruitful 540 665 509 632 637 555
# + M±S+ O author authorise 525 564 491 670 684 657+ M±S+ O hard hardly 519 532 504 583 520 531+ M±S+ O import important 593 638 566 585 681 595+ M±S+ O create creature 587 609 607 546 743 615+ M±S+ O patron patronise 571 627 556 601 755 635+ M±S+ O court courteous 629 649 538 565 594 617+ M±S+ O base basement 506 612 495 600 576 729± M+ S±O wealth fortune 536 570 608 574 563 625± M+ S±O prot gain 669 664 572 619 595 610± M+ S±O boss chief 557 560 533 495 562 532± M+ S±O ascend climb 602 662 601 540 639 669± M+ S±O wash cleanse 545 632 517 567 532 551± M+ S±O rule law 624 587 557 553 552 649± M+ S±O couch sofa 677 570 675 596 650 588± M+ S±O circle ring 594 567 565 530 542 600± M+ S±O ill sick 591 646 620 574 558 690± M+ S±O construct build 622 644 593 560 562 739± M+ S±O knock bump 649 566 552 496 589 636± M+ S±O battle fight 552 585 537 520 495 586± M+ S±O jacket coat 553 543 500 568 535 627± M+ S±O ban forbid 672 566 541 671 623 675± M+ S±O tremble shiver 584 622 546 628 636 575± M+ S±O honour glory 628 628 492 557 594 550± M+ S±O feeble weak 722 637 656 675 644 666± M+ S±O threat menace 578 578 539 622 623 530± M+ S±O scotch whisky 630 676 528 598 607 625± M+ S±O paper card 567 595 511 498 494 554± M+ S±O sob weep 715 672 560 659 632 610± M+ S±O pie tart 544 575 504 521 659 530± M+ S±O dart arrow 583 631 578 589 551 612± M+ S±O lane alley 546 538 522 598 606 679± M±S+ O elect electrode 706 682 790 585 820 792± M±S+ O enter enterprise 549 551 522 520 629 557± M±S+ O art artery 554 566 534 571 553 606± M±S+ O bell belly 573 559 555 530 611 594± M±S+ O bullet bulletin 518 605 595 559 593 590± M±S+ O car cardiac 586 556 545 545 522 576± M±S+ O tape tapestry 547 558 584 581 495 551± M±S+ O rub rubber 590 577 549 582 689 669± M±S+ O stride strident 607 645 628 684 668 672± M±S+ O tail tailor 559 570 547 485 525 573± M±S+ O stamp stampede 573 610 645 499 585 561± M±S+ O chap chapter 720 703 638 654 619 630± M±S+ O corn corner 639 609 522 580 629 595± M±S+ O harm harmony 540 625 571 603 642 601± M±S+ O mess message 666 629 577 547 618 587± M±S+ O sock socket 556 590 495 561 573 561± M±S+ O brand brandy 535 659 645 633 645 624± M±S+ O tent tentacle 538 579 499 530 590 526± M±S+ O demon demonstrate 536 615 558 640 645 554± M±S+ O colon colony 682 703 606 707 779 674
# ± M±S+ O accord accordion 565 627 555 604 672 635± M±S+ O intern international 719 659 637 680 857 787± M±S+ O dial dialect 631 636 519 596 604 554"""

In [ ]:
# rastle_exp1 = re.findall(r"([+±-])\s*M([+±-])\s*S([+±-])\s*O (\w+) (\w+)", s)
# rastle_exp1 = pd.DataFrame(rastle_exp1, columns=["M", "S", "O", "base", "inflected"])
# rastle_exp1["epen"] = rastle_exp1.apply(lambda x: x.inflected[len(x.base):], axis=1)
# rastle_exp1 = rastle_exp1.sort_values(["M", "S", "O", "epen"])

In [103]:
labels = state_space_spec.label_counts
labels = labels[labels >= 10]
labels = labels.index.tolist()

In [54]:
# suffixes = ["ment", "er", "ly", "less"]

In [88]:
inverse_phonemic_map = defaultdict(Counter)
for (label, _), row in cut_phonemic_forms.to_frame().iterrows():
    inverse_phonemic_map[row.description][label] += 1

In [ ]:
# er_forms = cut_phonemic_forms[cut_phonemic_forms.str.endswith("ER")] \
#     .groupby("label").filter(lambda xs: xs.name in labels and xs.iloc[0][:-2].strip() in inverse_phonemic_map).to_frame()

# er_forms["bases"] = er_forms.apply(lambda x: ",".join(x for x, _ in inverse_phonemic_map[x.description[:-2].strip()].most_common()), axis=1)
# er_forms = er_forms.reset_index()[["label", "bases", "description"]].drop_duplicates()
# er_forms.to_csv("er_forms.csv", index=False)

In [ ]:
# all = []
# for suffix in suffixes:
#     matches = [x for x in labels if x.endswith(suffix) and x[:-len(suffix)] in labels]
#     print(f"{suffix}: {len(matches)}")
#     if len(matches) > 20:
#         ret = pd.DataFrame(matches, columns=["inflected"])
#         ret["base"] = ret.inflected.str[:-len(suffix)]
#         ret["epen"] = suffix
#         all.append(ret)

# pd.concat(all).to_csv("suffixed_words.csv")

In [ ]:
er_forms = pd.read_csv("er_forms.csv")
er_forms = er_forms.rename(columns={"label": "inflected", "bases": "base", "description": "inflected_phones",
                                    "Morphologically related?": "morph_related"})
er_forms["morph_related"] = er_forms.morph_related.str.lower().map({"y": True, "n": False})
er_forms["base_phones"] = er_forms.inflected_phones.str[:-3]
print(er_forms.morph_related.value_counts())
er_forms = er_forms.dropna()
# remove any rows with , in the `base` field
er_forms = er_forms[~er_forms.base.str.contains(",")]
er_forms.morph_related.value_counts()

In [158]:
inflection_results_df = er_forms.copy()
inflection_results_df["inflection"] = "er"
inflection_results_df["base_idx"] = inflection_results_df.base.map({v: k for k, v in enumerate(state_space_spec.labels)})
inflection_results_df["inflected_idx"] = inflection_results_df.inflected.map({v: k for k, v in enumerate(state_space_spec.labels)})

In [ ]:
inflection_cross_instances = []
base_cross_instances = []

for _, row in tqdm(inflection_results_df.iterrows(), total=len(inflection_results_df)):
    inflected_instance_idxs = ss_spans.query(f"label == @row.inflected").instance_idx
    inflected_forms = cut_phonemic_forms.loc[row.inflected]
    for inflected_instance_idx in inflected_instance_idxs:
        if inflected_forms.loc[inflected_instance_idx] != row.inflected_phones:
            continue
        inflection_cross_instances.append({
            "inflection": row.inflection,
            "base": row.base,
            "inflected": row.inflected,
            "inflected_instance_idx": inflected_instance_idx,
            "inflected_phones": inflected_forms.loc[inflected_instance_idx]
        })

    base_instance_idxs = ss_spans.query(f"label == @row.base").instance_idx
    base_forms = cut_phonemic_forms.loc[row.base]
    for base_instance_idx in base_instance_idxs:
        if base_forms.loc[base_instance_idx] != row.base_phones:
            continue
        base_cross_instances.append({
            "inflection": row.inflection,
            "base": row.base,
            "inflected": row.inflected,
            "base_instance_idx": base_instance_idx,
            "base_phones": base_forms.loc[base_instance_idx]
        })

In [160]:
inflection_cross_instances_df = pd.DataFrame(inflection_cross_instances)
all_cross_instances = pd.merge(pd.DataFrame(base_cross_instances),
         inflection_cross_instances_df,
         on=["inflection", "base", "inflected"],
         how="outer")

# Now merge with type-level information.
all_cross_instances = pd.merge(inflection_results_df,
                               all_cross_instances,
                               on=["inflection", "base", "inflected", "base_phones", "inflected_phones"],
                               validate="1:m")

In [ ]:
all_cross_instances

In [ ]:
# # Add on random word pair baseline
# num_random_word_pairs = inflection_results_df.groupby("inflection").size().max()
# random_word_pairs = np.random.choice(len(list(labels)), size=(num_random_word_pairs, 2))
# random_word_pairs = pd.DataFrame(random_word_pairs, columns=["base_idx", "inflected_idx"])
# random_word_pairs["base"] = random_word_pairs.base_idx.map({i: l for i, l in enumerate(state_space_spec.labels)})
# random_word_pairs["inflected"] = random_word_pairs.inflected_idx.map({i: l for i, l in enumerate(state_space_spec.labels)})
# random_word_pairs["inflection"] = "random"
# random_word_pairs = random_word_pairs.set_index("inflection")
# random_word_pairs

In [163]:
# inflection_results_df = pd.concat([inflection_results_df, random_word_pairs])

## Save

In [ ]:
state_space_spec.to_hdf5(f"{output_dir}/state_space_spec.h5")

In [165]:
inflection_results_df.to_parquet(f"{output_dir}/inflection_results.parquet")
all_cross_instances.to_parquet(f"{output_dir}/all_cross_instances.parquet")